# Table of Contents
* 1. Import and review data  
        * 1.1 Import packages  
        * 1.2 Import data  
* 2. K-Nearest Neighbors
* 3. Random Forest
* 4. Gradient Boosting Machines - LightGBM
* 5. Support Vector Regression - RBF

      

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


In [2]:
#confirm the disk location is correct for importing
os.chdir('C:/Users/steve/Documents/springboard/Capstone2/data')
cwd = os.getcwd()
print(cwd)

C:\Users\steve\Documents\springboard\Capstone2\data


In [3]:
%%time
X_train = pd.read_csv('X_train.csv')
X_test = pd.read_csv('X_test.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

CPU times: total: 22.4 s
Wall time: 44.6 s


In [4]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(5605173, 166)
(5605173, 1)
(1401294, 166)
(1401294, 1)


# K-Nearest Neighbors

In [ ]:
%%time
from sklearn.neighbors import KNeighborsRegressor

knn=KNeighborsRegressor(n_neighbors=13)
knn.fit(X_train_sample,y_train_sample)

In [ ]:
%%time
from sklearn.model_selection import KFold, cross_val_score, RandomizedSearchCV
kf=KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {'n_neighbors':np.arange(1,15)}
knn = KNeighborsRegressor()
knn_cv= RandomizedSearchCV(knn,param_grid,cv=kf)
knn_cv.fit(X_train_sample,y_train_sample)

In [ ]:
print("Best Score:" + str(knn_cv.best_score_))
print("Best Parameters: " + str(knn_cv.best_params_))

In [ ]:
%%time
print(knn.score(X_train_sample,y_train_sample))

In [ ]:
%%time
print(knn.score(X_test_sample,y_test_sample))

In [ ]:
%%time
y_pred_knn = knn.predict(X_test_sample)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Calculate evaluation metrics
mae = mean_absolute_error(y_test_sample, y_pred_knn)
mse = mean_squared_error(y_test_sample, y_pred_knn)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_sample, y_pred_knn)

# Print evaluation metrics
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R2): {r2}')


# Random Forest

In [5]:
# Split the training and test data into a smaller sample
from sklearn.model_selection import train_test_split

sample_fraction = 0.2

X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=1-sample_fraction, random_state=42)
X_test_sample, _, y_test_sample, _ = train_test_split(X_test, y_test, test_size=1-sample_fraction, random_state=42)

In [6]:
%%time
print(X_train_sample.shape)
print(y_train_sample.shape)
print(X_test_sample.shape)
print(y_test_sample.shape)

(1121034, 166)
(1121034, 1)
(280258, 166)
(280258, 1)
CPU times: total: 0 ns
Wall time: 0 ns


In [7]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import randint


In [8]:
rf = RandomForestRegressor(random_state=42)

# Defining the parameter grid
param_dist = {
    'n_estimators': randint(50, 200),
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': randint(10, 50),
    'min_samples_split': randint(2, 10),
    'min_samples_leaf': randint(1, 10),
    'bootstrap': [True, False]}

In [9]:
# Setting up the RandomizedSearchCV
random_search_rf = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=5,  
    cv=5,  
    verbose=2,  
    random_state=42,
    n_jobs=-1)

In [10]:
%%time
# Fitting the model
random_search_rf.fit(X_train_sample, y_train_sample.values.ravel())


Fitting 5 folds for each of 5 candidates, totalling 25 fits
CPU times: total: 5min 6s
Wall time: 30min 39s


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=42),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EE71667450>,
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EE71C7C550>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EE71C7BF10>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001EE6FAB5D90>},
                   random_state=42, verbose=2)

In [11]:
print(f'Best Parameters: {random_search_rf.best_params_}')

Best Parameters: {'bootstrap': False, 'max_depth': 39, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 9, 'n_estimators': 70}


In [12]:
%%time
# Making predictions with the best estimator
best_model_rf = random_search_rf.best_estimator_





CPU times: total: 0 ns
Wall time: 0 ns


In [13]:
%%time
y_pred_rf = best_model_rf.predict(X_test_sample)

CPU times: total: 6.31 s
Wall time: 6.98 s


In [23]:
%%time
# Evaluating the model
mae_rf = mean_absolute_error(y_test_sample, y_pred_rf)
mse_rf = mean_squared_error(y_test_sample, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test_sample, y_pred_rf)

# Print evaluation metrics
print(f'Mean Absolute Error (MAE): {mae_rf}')
print(f'Mean Squared Error (MSE): {mse_rf}')
print(f'Root Mean Squared Error (RMSE): {rmse_rf}')
print(f'R-squared (R2): {r2_rf}')


Mean Absolute Error (MAE): 0.6780843160333464
Mean Squared Error (MSE): 0.8558391544551969
Root Mean Squared Error (RMSE): 0.9251157519225348
R-squared (R2): 0.14121759948016466
CPU times: total: 0 ns
Wall time: 25.5 ms


In [15]:
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [48]:
# Define the objective function to optimize
def rf_eval(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    """
    Trains a Random Forest with given hyperparameters and returns the cross-validated score.
    """
    # Convert float inputs to int (as hyperparameters must be integers)
    n_estimators = int(n_estimators)
    max_depth = int(max_depth)
    min_samples_split = int(min_samples_split)
    min_samples_leaf = int(min_samples_leaf)
    
    
    # Create Model
    rfbo = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=-1,
        
    )
    
    rfbo.fit(X_train_sample, y_train_sample.values.ravel())
    y_pred_rfbo = rfbo.predict(X_test_sample)
    mse = mean_squared_error(y_test_sample, y_pred_rfbo)
    return -mse  


In [49]:
# Define the parameter search space
pbounds = {
    "n_estimators": (50, 200),  
    "max_depth": (5, 50), 
    "min_samples_split": (2, 10), 
    "min_samples_leaf": (1, 10),
}

In [50]:
# Set up the optimizer
optimizer = BayesianOptimization(
    f=rf_eval,  
    pbounds=pbounds, 
    random_state=42,
    verbose=2
)

In [51]:
%%time
# Run optimization
optimizer.maximize(init_points=4, n_iter=8)

# Print best parameters
print("Best hyperparameters found:", optimizer.max)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | -0.8693   | 21.85     | 9.556     | 7.856     | 139.8     |
| 2         | -0.9      | 12.02     | 2.404     | 2.465     | 179.9     |
| 3         | -0.8537   | 32.05     | 7.373     | 2.165     | 195.5     |
| 4         | -0.8942   | 42.46     | 2.911     | 3.455     | 77.51     |
| 5         | -0.8537   | 32.8      | 7.93      | 2.832     | 194.4     |
| 6         | -0.952    | 49.16     | 2.0       | 3.286     | 198.5     |
| 7         | -0.8648   | 23.67     | 7.746     | 9.517     | 195.0     |
| 8         | -0.8554   | 29.82     | 9.785     | 2.826     | 180.5     |
| 9         | -0.8512   | 37.13     | 8.821     | 4.942     | 163.8     |
| 10        | -0.8508   | 45.45     | 8.885     | 8.115     | 150.6     |
| 11        | -0.8814   | 49.84     | 2.677     | 9.582     | 162.3     |
| 12        | -0.8991   | 32.71     | 

In [55]:
%%time
# Extract the best parameters
best_params_rf = optimizer.max['params']
best_params_rf['n_estimators'] = int(best_params_rf['n_estimators'])
best_params_rf['max_depth'] = int(best_params_rf['max_depth'])
best_params_rf['min_samples_split'] = int(best_params_rf['min_samples_split'])
best_params_rf['min_samples_leaf'] = int(best_params_rf['min_samples_leaf'])

print(best_params_rf)

{'max_depth': 45, 'min_samples_leaf': 8, 'min_samples_split': 8, 'n_estimators': 150}
CPU times: total: 0 ns
Wall time: 1.12 ms


{'max_depth': 45, 'min_samples_leaf': 8.88475769057052, 'min_samples_split': 8.114762421844707, 'n_estimators': 150}
